# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [ ]:
import pandas as pd
import numpy as np

# 1. Carregando a base de dados
try:
    df = pd.read_csv('demo01.csv')
    print("Base de dados 'demo01.csv' carregada com sucesso!")
except FileNotFoundError:
    print("Erro: O arquivo 'demo01.csv' não foi encontrado. Certifique-se de que ele está no mesmo diretório do seu script ou forneça o caminho completo.")
    print("Você pode baixar o dataset de: https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction")
    # Se o arquivo não for encontrado, encerraremos o script para evitar erros posteriores.
    exit() # <--- Se este exit() for acionado, as variáveis df e df_dummies não serão criadas!

# ... (restante do código da Questão 1)
print("\nPrimeiras 5 linhas do dataset:")
print(df.head())

print("\nInformações gerais do dataset:")
df.info()

print("\nEstatísticas descritivas do dataset:")
print(df.describe())

# ... (continuação do código da Questão 1 para tratamento de missings)
for column in df.columns:
    if df[column].dtype == 'object':
        if df[column].isnull().sum() > 0:
            mode_value = df[column].mode()[0]
            df[column].fillna(mode_value, inplace=True)
            print(f"Missing values na coluna '{column}' (categórica) preenchidos com a moda: {mode_value}")
    else:
        if df[column].isnull().sum() > 0:
            median_value = df[column].median()
            df[column].fillna(median_value, inplace=True)
            print(f"Missing values na coluna '{column}' (numérica) preenchidos com a mediana: {median_value}")

print("\nVerificação de missings após tratamento:")
print(df.isnull().sum().sum())
if df.isnull().sum().sum() == 0:
    print("Todos os valores ausentes foram tratados.")
else:
    print("Ainda existem valores ausentes. Revise a estratégia de tratamento.")

if 'mau' in df.columns:
    print("\nDistribuição da variável resposta 'mau':")
    print(df['mau'].value_counts())
    print("\nProporção da variável resposta 'mau':")
    print(df['mau'].value_counts(normalize=True) * 100)
else:
    print("\nA coluna 'mau' (variável resposta) não foi encontrada na base de dados.")

print("\n--- Fim da Questão 1 ---")

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [ ]:
# 2. Vamos montar um metadados

print("\n--- Questão 2: Montando Metadados ---")

# 1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
metadados = pd.DataFrame({
    'nome_variavel': df.columns,
    'tipo_dado': df.dtypes.values
})

# 2. Adicione uma coluna nesse dataframe chamada "qnt_categoria"
# e coloque nela o número de categorias correspondentes de cada variável.
# Dica 1: inicie uma lista vazia.
# Dica 2: faça um for ao longo dos nomes das variáveis.
# Dica 3: conte o número de categorias dessa variável.
# Dica 4: acumule essa informação de 3. na lista que você criou em 1.
# Dica 5: No final, essa lista pode ser atribuída à nossa variável.

qnt_categoria = []
for col in df.columns:
    if df[col].dtype == 'object' or df[col].nunique() < 20: # Consideramos menos de 20 valores únicos como categórica para fins de contagem de categorias
        qnt_categoria.append(df[col].nunique())
    else:
        qnt_categoria.append(np.nan) # Para variáveis contínuas, não faz sentido contar categorias

metadados['qnt_categoria'] = qnt_categoria

# Adicionar a coluna 'categoria' (qualitativa/quantitativa)
metadados['categoria'] = metadados.apply(lambda row: 'qualitativa' if row['tipo_dado'] == 'object' or (row['tipo_dado'] != 'object' and row['qnt_categoria'] is not np.nan and row['qnt_categoria'] < 20) else 'quantitativa', axis=1)


print("\nDataframe de Metadados:")
print(metadados)

print("\n--- Fim da Questão 2 ---")

#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [ ]:
# 3. Crie variáveis dummy

print("\n--- Questão 3: Criação de Variáveis Dummy ---")

df_dummies = df.copy() # <--- df_dummies é criado aqui! Isso pressupõe que 'df' já existe.

object_columns = df_dummies.select_dtypes(include='object').columns
print(f"\nVariáveis categóricas a serem transformadas em dummy: {list(object_columns)}")

if len(object_columns) > 0:
    df_dummies = pd.get_dummies(df_dummies, columns=object_columns, drop_first=True, dtype=int)
    print("\nDataframe após a criação de variáveis dummy:")
    print(df_dummies.head())
    print(f"\nFormato do dataframe após criação das dummies: {df_dummies.shape}")
else:
    print("\nNão há variáveis do tipo 'object' para transformar em dummy.")

print("\nNovas colunas criadas (apenas as 5 primeiras para visualização):")
print(df_dummies.columns.tolist()[:5])

print("\n--- Fim da Questão 3 ---")

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [ ]:
# 4. Qual variável é mais poderosa?

print("\n--- Questão 4: Análise de Poder Preditivo ---")

# Verificando a existência das colunas
required_cols = ['mau', 'posse_email', 'posse_de_veiculo']
if not all(col in df.columns for col in required_cols):
    print(f"Erro: Uma ou mais colunas necessárias ({required_cols}) não foram encontradas no dataframe.")
else:
    print("\nTabela cruzada entre 'mau' e 'posse_email':")
    crosstab_email = pd.crosstab(df['posse_email'], df['mau'], normalize='index') * 100
    print(crosstab_email)

    print("\nTabela cruzada entre 'mau' e 'posse_de_veiculo':")
    crosstab_veiculo = pd.crosstab(df['posse_de_veiculo'], df['mau'], normalize='index') * 100
    print(crosstab_veiculo)

    # Análise para determinar qual é mais poderosa
    # Uma variável é mais poderosa se houver uma maior diferença nas proporções de 'mau=1'
    # entre as categorias da variável preditora.

    # Exemplo de análise:
    # Para 'posse_email':
    # Olhe a diferença entre a % de 'mau=1' quando posse_email é 0 e quando posse_email é 1.
    # Ex: abs(crosstab_email.loc[0, 1] - crosstab_email.loc[1, 1])

    # Para 'posse_de_veiculo':
    # Olhe a diferença entre a % de 'mau=1' quando posse_de_veiculo é 0 e quando posse_de_veiculo é 1.
    # Ex: abs(crosstab_veiculo.loc[0, 1] - crosstab_veiculo.loc[1, 1])

    # Quanto maior a diferença, maior o poder preditivo da variável isoladamente.

    # Calculando a diferença nas proporções de mau=1
    diff_email = abs(crosstab_email.loc[0, 1] - crosstab_email.loc[1, 1])
    diff_veiculo = abs(crosstab_veiculo.loc[0, 1] - crosstab_veiculo.loc[1, 1])

    print(f"\nDiferença na proporção de 'mau=1' para 'posse_email': {diff_email:.2f}%")
    print(f"Diferença na proporção de 'mau=1' para 'posse_de_veiculo': {diff_veiculo:.2f}%")

    if diff_email > diff_veiculo:
        print("\n**Conclusão: 'posse_email' parece ser a variável mais poderosa para prever 'mau = 1' com base na diferença de proporções.**")
    elif diff_veiculo > diff_email:
        print("\n**Conclusão: 'posse_de_veiculo' parece ser a variável mais poderosa para prever 'mau = 1' com base na diferença de proporções.**")
    else:
        print("\n**Conclusão: Ambas as variáveis têm poder preditivo similar.**")

print("\n--- Fim da Questão 4 ---")

#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [3]:
# 5. Salve a base

print("\n--- Questão 5: Salvando a Base ---")

output_filename = 'demo01_processed.csv'

try:
    if 'df_dummies' in globals() and isinstance(df_dummies, pd.DataFrame):
        df_to_save = df_dummies
        print("Salvando o DataFrame 'df_dummies' (com correções e variáveis dummy).")
    elif 'df' in globals() and isinstance(df, pd.DataFrame):
        df_to_save = df
        print("Salvando o DataFrame 'df' (com correções de missings, sem dummies, pois 'df_dummies' não foi encontrado ou é inválido).")
    else:
        print("Erro: Nenhum DataFrame válido ('df_dummies' ou 'df') foi encontrado para salvar.")
        df_to_save = None

    if df_to_save is not None:
        df_to_save.to_csv(output_filename, index=False)
        print(f"\nBase de dados processada salva com sucesso como '{output_filename}'")

except Exception as e:
    print(f"\nErro ao salvar a base de dados: {e}")

print("\n--- Fim da Questão 5 ---")


--- Questão 5: Salvando a Base ---
Erro: Nenhum DataFrame válido ('df_dummies' ou 'df') foi encontrado para salvar.

--- Fim da Questão 5 ---
